In [1]:
import numpy as np
import math
from decimal import *
import time

In [16]:
N = 128
x = np.linspace(0.,1.,N)
stepsize = x[1] - x[0]
alpha = 0.5

def f(x):
    """ Test square root function for DFFT convolution. """
    return np.sqrt(x)

def isInteger(n):
    """Return True if argument is a whole number, False if argument has a fractional part.

    Note that for values very close to an integer, this test breaks. During
    superficial testing the closest value to zero that evaluated correctly
    was 9.88131291682e-324. When dividing this number by 10, Python 2.7.1 evaluated
    the result to zero"""

    if n%2 == 0 or (n+1)%2 == 0:
        return True
    return False

def poch(a,n):
    # Returns the Pochhammer symbol (a)_n.
    
    # First, check if 'a' is a real number (this is currently only working for reals).
    if np.iscomplex(a):
        raise ValueError('Value "a" must be a real number.')
    elif n < 0:
        raise ValueError('Value "n" must be a positive integer.')
    elif isInteger(n) == False:
        raise ValueError('Value "n" must be an integer.')
    
    # Compute the Pochhammer symbol.
    if n == 0:
        poch = 1
        return poch
    else:
        poch = 1
        for j in range(n):
            poch *= (a + j)
        return poch

def GLcoeffs(alpha,n):
    # Computes the coefficient array of size n.
    if n < 0:
        raise ValueError('Value "n" must be positive.')
    elif isInteger(n) == False:
        raise ValueError('Value "n" must be an integer.')
        
    else:
        #GL_filter = np.zeros(n,)
        GL_filter = [poch(-alpha,j)/np.math.factorial(j) for j in range(n)]
        #for j in range(n):
        #    GL_filter[j] = poch(-alpha,j)/np.math.factorial(j)
            #print(GL_filter[j])
    return GL_filter

F = f(x)
DD = GLcoeffs(alpha,N)

start = time.time()
FF = np.fft.rfft(F)
DDFF = np.fft.rfft(DD)

result = stepsize**(-alpha)*np.fft.irfft(FF*DDFF)
elapsed = time.time() - start

experiment = result[len(result)-1]
truth = np.sqrt(np.pi)/2

abserror = abs(experiment - truth)
relerror = abs((experiment-truth)/truth)
print(abserror)
print(relerror)
print("{0:.15f}".format(elapsed) + ' seconds')

4.11555563085e-05
4.64390723487e-05
0.001000881195068 seconds


It looks like this is the way to do it. It's extremely fast and just as accurate as the brute force method. Plus, this code computes the GL differintegral on the entire domain of differintegration. PROBLEM: for large $N$, this code has integer overflow in the Pochhammer symbol function. The factorial function also can't handle large integer values. I may have to take the logarithm to reduce the size of the binomial coefficients and then exponentiate the sum of logs.

I still need to compare this to the matrix multiplication method (vector multiplication is not very accurate).